In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:35823")
client

<Client: 'tcp://127.0.0.1:35823' processes=8 threads=32, memory=125.87 GiB>

In [2]:
client.restart()

<Client: 'tcp://127.0.0.1:36609' processes=0 threads=0, memory=0 B>

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import os.path as op
import os
import gsw
from xgcm.grid import Grid
from xmitgcm import open_mdsdataset
from fastjmd95 import rho as densjmd95
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
ddir = '/tank/chaocean/qjamet/RUNS/ORAR/reruns2/'
gdir = '/tank/chaocean/grid_chaO/gridMIT_update1/'
savedir = '/tank/spectre/tuchida/chaocean/Snap/'

In [ ]:
grav = 9.81
nensembs = 5
Kh = 20.
K4 = 1e10
Kr = 1e-5
thetaMax = 1e20
rhoConst = 9.998e2
# Cp = 3.994e3
Cp = 3989.244953
# years = np.arange(2008,2013,dtype=int)
ySstart = -15
yNstart = 15
yend = 50
dnf = 2160
nfile = 73

In [ ]:
xchunk = 100
ychunk = 100
# dCs = 1e-5
# Cs = xr.open_dataset(op.join(savedir,"Max-snap-sound-speed_1967.nc")).Cs
# Cs

In [ ]:
years = np.arange(1967,1968,dtype=int)
ntimes = np.arange(1419120,1576800,dnf).reshape(len(years),nfile)
dd = 63  # 
yystart = 0
ntimes[yystart,dd:]

# Snapshots

In [ ]:
nmembs = np.arange(100,148)
xchunk = 100
ychunk = 100

yy = yystart

for itime in ntimes[yystart,dd+1:]:
    for nmemb in nmembs:
        if nmemb == 100:
            ds = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                         % (nmemb)), 
                                 grid_dir=gdir,
                                 prefix=['snap_diagOcn'], delta_t=2e2,
                                 iters=itime, 
                                ).isel(time=0).chunk({'YC':ychunk,'YG':ychunk,
                                                      'XC':xchunk,'XG':xchunk})
        else:
            ds = xr.concat([ds, open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                                        % (nmemb)), 
                                                grid_dir=gdir,
                                                prefix=['snap_diagOcn'], delta_t=2e2,
                                                iters=itime, 
                                               ).isel(time=0).chunk({'YC':ychunk,'YG':ychunk,
                                                                     'XC':xchunk,'XG':xchunk})
                           ], dim='nmemb')

    ds.coords['nmemb'] = ('nmemb',nmembs)
    
    grid = Grid(ds, periodic=['X'])

    pres = grav*np.abs(-ds.Z)
    theta = ds.THETA.where(ds.maskC!=0.)
    salt = ds.SALT.where(ds.maskC!=0.)
    
    grid = Grid(ds, periodic=['X'])
    # rho_til = grid.interp(rhoConst*(1. - grav*(-grid.cumsum((Cs+dCs)**-2*ds.drF.load(), 'Z',
    #                                                         boundary='extend')
    #                                           )),
    #                       'Z', boundary='extend'
    #                      )
    rho = xr.apply_ufunc(densjmd95, salt, theta, 
                         pres, 
                         dask='parallelized', output_dtypes=[float,]
                        )
    # b = -grav * (rho - rho_til) / rhoConst
    b = -grav * (rho - rhoConst) / rhoConst
    
    b.to_dataset(name='BUOY').to_zarr(op.join(savedir,'%04d/%010d/BUOY.zarr' 
                                              % (years[0],itime)),
                                      mode='w')
    print(itime)

# Ensemble mean

In [8]:
nmembs = np.arange(100,148)
xchunk = 100
ychunk = 100

for itime in ntimes[yystart,dd:]:
    ds = open_mdsdataset(op.join(ddir,'ensm/run1967/ocn/'), 
                         grid_dir=gdir,
                         prefix=['snap_diagOcn'], delta_t=2e2,
                         iters=itime, 
                        ).isel(time=0).chunk({'YC':ychunk,'YG':ychunk,
                                              'XC':xchunk,'XG':xchunk})
    
    grid = Grid(ds, periodic=['X'])

    pres = grav*np.abs(-ds.Z)
    theta = ds.THETA.where(ds.maskC!=0.)
    salt = ds.SALT.where(ds.maskC!=0.)
    
    grid = Grid(ds, periodic=['X'])
    # rho_til = grid.interp(rhoConst*(1. - grav*(-grid.cumsum((Cs+dCs)**-2*ds.drF.load(), 'Z',
    #                                                         boundary='extend')
    #                                           )),
    #                       'Z', boundary='extend'
    #                      )
    rho = xr.apply_ufunc(densjmd95, salt, theta, 
                         pres, 
                         dask='parallelized', output_dtypes=[float,]
                        )
    # b = -grav * (rho - rho_til) / rhoConst
    b = -grav * (rho - rhoConst) / rhoConst
    
    b.to_dataset(name='BUOY').to_zarr(op.join(savedir,'%04d/ensm/%010d/BUOY.zarr' 
                                              % (years[0],itime)),
                                      mode='w')
    print(itime)

1419120
1421280
1423440
1425600
1427760
1429920
1432080
1434240
1436400
1438560
1440720
1442880
1445040
1447200
1449360
1451520
1453680
1455840
1458000
1460160
1462320
1464480
1466640
1468800
1470960
1473120
1475280
1477440
1479600
1481760
1483920
1486080
1488240
1490400
1492560
1494720
1496880
1499040
1501200
1503360
1505520
1507680
1509840
1512000
1514160
1516320
1518480
1520640
1522800
1524960
1527120
1529280
1531440
1533600
1535760
1537920
1540080
1542240
1544400
1546560
1548720
1550880
1553040
1555200
1557360
1559520
1561680
1563840
1566000
1568160
1570320
1572480
1574640
